<font color='red'> Penalty kick application based on 'Professionals Play Minimax' by Ignacio Palacios-Huerta (2003). </font>

### Imports

In [1]:
from bokeh.io import show, output_notebook, save, output_file
from bokeh.models import Plot, TapTool, ColumnDataSource, LabelSet, StaticLayoutProvider, Circle, MultiLine
from bokeh.models.widgets import Div
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.plotting import figure
from bokeh.models.renderers import GraphRenderer, GlyphRenderer
from bokeh.layouts import layout, row, column, gridplot
from bokeh.models import CustomJS, Button, DataTable, TableColumn, Rect
from bokeh.models.glyphs import Text
from bokeh.events import ButtonClick

In [2]:
output_notebook()

Loading BokehJS ...

### Define Figure and Sprites

In [3]:
p = figure(tools="", 
           toolbar_location=None, 
           title='FIFA 2020 Penalty Simulator',
           plot_width=600, plot_height=480, 
           x_range=(0, 100), y_range=(0, 90))
p.title.text_font_size = '15pt'

# hide axes and gridlines
p.xaxis.visible = False
p.yaxis.visible = False
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.outline_line_color = None

# background color
p.background_fill_color = "green"

# goal posts and lines
p.multi_line([[24, 76, 78, 22, 24], [3,  12, 88, 97, 3 ], [34, 34, 66, 66]],
             [[63, 63, 47, 47, 63], [15, 63, 63, 15, 15], [63, 82, 82, 63]],
             color=["lightgreen", "lightgreen", "whitesmoke"],
             alpha=[1, 1, 1], line_width=4)

# striker box
p.quadratic(33, 15, 67, 15, 50, 2, color='lightgreen', line_width=4)

# goalie sprite 
goalie_head = Circle(x=50, y=69, fill_color='red', line_width=2, size=17)
goalie_body = Rect(x=50, y=65, width=3, height=4, angle=0, fill_color='red', line_width=2)
p.add_glyph(goalie_head)
p.add_glyph(goalie_body)

# ball 
ball = Circle(x=50, y=13, fill_color='whitesmoke', line_width=2, size=17)
p.add_glyph(ball);

# striker sprite 
p.add_glyph(Circle(x=50, y=16, fill_color='lightblue', line_width=2, size=21)) # head
p.add_glyph(Rect(x=50, y=11, width=4, height=6, angle=0, fill_color='lightblue', line_width=2)); #body

### Define Labels

In [4]:
scr_text = ColumnDataSource({'x': [2, 70, 2, 14, 14],
                             'y': [86, 86, 5, 40, 32],
                             'text': ['Rounds played: 0',
                                      'Total score: 0',
                                      '',
                                      '',
                                      '']})

labels = Text(x="x", y="y", text='text', text_color="whitesmoke",
              text_font_size= '15pt', x_offset=0, y_offset=+9,
              text_baseline='ideographic', text_align='left')

p.add_glyph(scr_text, labels);

### Define State Variables as Divs

In [5]:
nround = Div(text='0', visible=False) # total number of rounds completed
score = Div(text='0', visible=False) # current score
freq_left_foot = Div(text='0,0,0', visible=False) # frequency of left, middle, and right kicks (left-footed)
freq_right_foot = Div(text='0,0,0', visible=False) # frequency of left, middle, and right kicks (right-footed)
kicker_foot = Div(text='', visible=False) # current footedness of the kicker
kicker_kick = Div(text='', visible=False) # current direction the kicker will kick

### Define Buttons

In [6]:
# start and next buttons
b_start = Button(label="Begin", button_type="success", sizing_mode= 'scale_width', width_policy='fit')
b_next = Button(label="Next round", button_type="success", sizing_mode= 'scale_width', width_policy='fit', disabled=True)

# right and left footed buttons
b_fl = Button(label="Left-Footed", button_type="success", sizing_mode= 'scale_width', width_policy='fit', disabled=True)
b_fr = Button(label="Right-Footed", button_type="success", sizing_mode= 'scale_width', width_policy='fit', disabled=True)

# right, middle, and left kick buttons
bl = Button(label="Left", button_type="success", sizing_mode= 'scale_width', width_policy='fit', disabled=False, visible=False)
bm = Button(label="Middle", button_type="success", sizing_mode= 'scale_width', width_policy='fit', disabled=False, visible=False)
br = Button(label="Right", button_type="success", sizing_mode= 'scale_width', width_policy='fit', disabled=False, visible=False)

# shoot
b_shoot = Button(label="SHOOT!", button_type="success", sizing_mode= 'scale_width', width_policy='fit', disabled=False, visible=False)

### Define Callbacks

In [7]:
# next button
b_next_code = """
b_shoot.visible = false;

b_fr.visible = true;
b_fl.visible = true;
b_fr.disabled = false;
b_fl.disabled = false;

bl.visible = false;
bm.visible = false;
br.visible = false;

b_next.disabled = true;

txt.data['text'][2] = ''
txt.data['text'][3] = 'Choose a right- or left footed kicker!'
txt.change.emit()  

ball.x = 50
ball.y = 13
goalie_head.x = 50
goalie_body.x = 50
"""
b_next_click = CustomJS(args=dict(b_start=b_start, b_fl=b_fl, b_fr=b_fr, bl=bl, bm=bm, br=br,
                                  txt=scr_text, b_shoot=b_shoot, b_next=b_next, ball=ball,
                                  goalie_head=goalie_head, goalie_body=goalie_body),
                        code=b_next_code)
b_next.js_on_click(b_next_click)

In [8]:
# start button
b_start_code = b_next_code + """
if (b_start.label == 'Begin') {
    b_start.label = 'Restart'
} else {
    txt.data['text'][0] = 'Rounds played: 0'
    txt.data['text'][1] = 'Total score: 0'
    txt.data['text'][2] = ''
    txt.data['text'][3] = ''
    txt.data['text'][4] = ''
    nround.text='0'
    score.text='0'
    freq_left_foot.text='0,0,0'
    freq_right_foot.text='0,0,0'
    kicker_foot.text=''
    kicker_kick.text=''
    txt.change.emit() 
    b_start.label = 'Begin'
}
"""
b_start_click = CustomJS(args=dict(b_start=b_start, b_fl=b_fl, b_fr=b_fr, bl=bl, bm=bm, br=br,txt=scr_text,
                                   kicker_kick=kicker_kick, kicker_foot=kicker_foot, freq_left_foot=freq_left_foot, 
                                   b_next=b_next,freq_right_foot=freq_right_foot, b_shoot=b_shoot, nround=nround, 
                                   score=score, ball=ball, goalie_head=goalie_head, goalie_body=goalie_body),
                         code=b_start_code)
b_start.js_on_click(b_start_click)

In [9]:
# select the foot type of striker buttons
def b_f_click(foot):
    code = """
    b_fl.disabled = true;
    b_fr.disabled = true;
    b_fl.visible = false;
    b_fr.visible = false;
    bl.visible = true;
    bm.visible = true;
    br.visible = true;
    txt.data['text'][3] = 'Choose where to kick!'
    txt.data['text'][2] = '"""+foot+"""-footed kicker.'
    kicker_foot.text= '"""+foot+"""'
    txt.change.emit()
    """
    return CustomJS(args=dict(b_start=b_start, b_fl=b_fl, b_fr=b_fr, bl=bl, bm=bm, br=br, txt=scr_text,
                              kicker_foot=kicker_foot),
                    code=code)
b_fl.js_on_click(b_f_click('Left'))
b_fr.js_on_click(b_f_click('Right'))

In [10]:
# select where the kicker will kick buttons
def b_kick_click(kick):
    code = """
    b_shoot.visible = true;
    bl.visible = false;
    bm.visible = false;
    br.visible = false;
    txt.data['text'][3] = ''
    var current = txt.data['text'][2]
    txt.data['text'][2] = current + ' Kicking """+kick+""".'
    kicker_kick.text= '"""+kick+"""'
    txt.change.emit()
    b_shoot.disabled = false;
    """
    return CustomJS(args=dict(b_start=b_start, b_fl=b_fl, b_fr=b_fr, bl=bl, bm=bm, br=br, txt=scr_text,
                              kicker_foot=kicker_foot, kicker_kick=kicker_kick, b_shoot=b_shoot),
                    code=code)

bl.js_on_click(b_kick_click('Left'))
bm.js_on_click(b_kick_click('Middle'))
br.js_on_click(b_kick_click('Right'))

In [11]:
# shoot button
b_shoot_code = """
// Define probability matrix
var p = {'Right' : {'LeftLeft' : 0.55,
                    'LeftMiddle' : 0.65,
                    'LeftRight' : 0.93,
                    'MiddleLeft' : 0.74,
                    'MiddleMiddle' : 0.60,
                    'MiddleRight' : 0.72,
                    'RightLeft' : 0.95,
                    'RightMiddle' : 0.73,
                    'RightRight' : 0.70},
         'Left' :  {'LeftLeft' : 0.67,
                    'LeftMiddle' : 0.70,
                    'LeftRight' : 0.96,
                    'MiddleLeft' : 0.74,
                    'MiddleMiddle' : 0.60,
                    'MiddleRight' : 0.72,
                    'RightLeft' : 0.87,
                    'RightMiddle' : 0.65,
                    'RightRight' : 0.61}}

// Choose best action
var freq
if (kicker_foot.text == 'Right') {
    freq = freq_right_foot.text.split(',').map(Number);
} else {
    freq = freq_left_foot.text.split(',').map(Number);
}

var kicker_action = 'Left'
var expected = (freq[0]*p[kicker_foot.text]['LeftLeft'] + 
                freq[1]*p[kicker_foot.text]['MiddleLeft'] +
                freq[2]*p[kicker_foot.text]['RightLeft'])   
var actions = ["Left", "Middle", "Right"]
for (var i = 0; i < 3; i++) {
    var val = (freq[0]*p[kicker_foot.text]['Left'+actions[i]] + 
               freq[1]*p[kicker_foot.text]['Middle'+actions[i]] + 
               freq[2]*p[kicker_foot.text]['Right'+actions[i]])
    if (val < expected) {
        kicker_action = actions[i]
        expected = val
    }
}    

// Determine if goal
var goal = 1
if (Math.random() > p[kicker_foot.text][kicker_kick.text+kicker_action]) {
    goal = -1
}

// Animate ball and goalie
ball.x = {'Left' : 40, 'Middle' : 50, 'Right' : 60}[kicker_kick.text]
ball.y = 63
goalie_head.x = {'Left' : 40, 'Middle' : 50, 'Right' : 60}[kicker_action]
goalie_body.x = {'Left' : 40, 'Middle' : 50, 'Right' : 60}[kicker_action]

// Add to frequency history
var dict = {'Left'  : 0,
            'Middle' : 1,
            'Right'   : 2}
if (kicker_foot.text == 'Right') {
    var freq = freq_right_foot.text.split(',')
    freq[dict[kicker_kick.text]] = parseInt(freq[dict[kicker_kick.text]]) + 1
    freq_right_foot.text = freq.toString()
} else {
    var freq = freq_left_foot.text.split(',')
    freq[dict[kicker_kick.text]] = parseInt(freq[dict[kicker_kick.text]]) + 1
    freq_left_foot.text = freq.toString()
}

// Update text
var n = (parseInt(nround.text) + 1)
nround.text = n.toString()
txt.data['text'][0] = 'Rounds played: ' + n

if (goal == 1) {
    txt.data['text'][3] = 'GOAL!'
} else {
    txt.data['text'][3] = 'Blocked'
}

var s = parseInt(score.text) + goal
score.text = s.toString()
txt.data['text'][1] = 'Total score: ' + s

txt.change.emit()

// Allow starting next round
b_shoot.disabled = true;
b_next.disabled = false;
"""
b_shoot_click = CustomJS(args=dict(b_start=b_start, b_fl=b_fl, b_fr=b_fr, txt=scr_text, kicker_kick=kicker_kick,
                                   kicker_foot=kicker_foot, freq_left_foot=freq_left_foot, b_next=b_next,
                                   freq_right_foot=freq_right_foot, b_shoot=b_shoot, nround=nround, score=score,
                                   ball=ball, goalie_head=goalie_head, goalie_body=goalie_body),
                         code=b_shoot_code)
b_shoot.js_on_click(b_shoot_click)

### Format Layout and Show

In [12]:
buttons_bot = row(b_start, nround, b_fl, b_fr, b_shoot, bl, bm, br,
                  score, freq_left_foot, freq_right_foot, kicker_foot, 
                  kicker_kick, b_next, max_width=600, sizing_mode='stretch_both')
grid = gridplot([[p], [buttons_bot]], plot_width=600, plot_height=480)
show(grid)

### Create HTML File

In [13]:
output_file("penalty_kick_game.html")
save(grid);